In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('src'), '..')))

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import wandb

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr

seed_value = 42
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)
generator = torch.Generator()
generator.manual_seed(seed_value)
torch.backends.cudnn.deterministic = True

from functools import partial

from src.Trainer.Trainer import Trainer
from src.Losses.Focal_loss import FocalLoss
from src.Models.Model_class import Model_class
from src.Losses.Loss_class import Loss_class
from src.DatasetsClasses.DatasetClass import TableDatasetDF

from src.Models.Autoenc_simpl import Autoencoder
from src.Models.Classifier import Simple_classifier
from src.Models.JoinedModel import JoinedModel

from src.DatasetsClasses.EncDataset import EncoderDataset
from src.Losses.Loss_class_encoder import Loss_class_encoder

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
HIDDEN_PARAM = 128
LATENT_REPR = 10

BATCH_SIZE = 1024

In [3]:
df = pd.read_csv('../data/df_to_enc.csv')

In [4]:
def prepare_data_for_test(X_data, y_data, test_ratio):
    # Prepare dataset for testing
    X_train, X_test, y_train, y_test = train_test_split(X_data,
                                                     y_data,
                                                     shuffle = True,
                                                     stratify = y_data,
                                                     random_state = 42,
                                                     test_size = test_ratio)
    
    return X_train, X_test, y_train, y_test

In [5]:
def prepare_data_for_enc(X_data, y_data, autoenc_requared):

    # Check that amount rows for enc less than length of data
    if autoenc_requared >= len(X_data):
        raise ValueError("The number of rows for autoencoder more than amount of X_train data")
    
    autoenc_ratio = autoenc_requared/len(X_data)
    
    X_to_enc, X_to_clas,\
    y_to_enc, y_to_clas = train_test_split(X_data,
                                           y_data,
                                           shuffle = True,
                                           stratify = y_data,
                                           random_state = 42,
                                           train_size = autoenc_ratio)
    
    X_train, X_test = train_test_split(X_to_enc,
                                       shuffle = True,
                                       random_state = 42,
                                       train_size = 0.9)
    
    return X_train, X_test, X_to_clas, y_to_clas

In [6]:
def prepare_data_for_classif(X_data, y_data, classif_requared):

    # Prepare dataset for encoder
    if classif_requared >= len(X_data):
        raise ValueError("The number of rows for classifier more than amount of X_train data")
    classif_ratio = classif_requared / len(X_data)
    
    X_train, X_test,\
    y_train, y_test = train_test_split(X_data,
                                       y_data,
                                       shuffle = True,
                                       stratify = y_data,
                                       random_state = 42,
                                       train_size = classif_ratio)
    
    return X_train, y_train

In [7]:
def make_dataloader(*data, encoder_data = False):
    
    if len(data) > 1:
        data_list = [data[i] for i in range(len(data))]
        dataset = pd.concat(data_list, axis = 1)
    else:
        dataset = data[0]
    if encoder_data == False:
        dataset = TableDatasetDF(dataset)
        dataloader = DataLoader(
            dataset,
            batch_size=BATCH_SIZE, 
            shuffle=True,
            generator=generator
        )
    else:
        dataset = EncoderDataset(dataset)
        dataloader = DataLoader(
            dataset,
            batch_size=BATCH_SIZE, 
            shuffle=True,
            generator=generator
        )

    return dataloader

In [8]:
def prepare_data(X_data, y_data, test_ratio, autoenc_requared, classif_requared):
    X_train, X_test, y_train, y_test = prepare_data_for_test(X_data, y_data, test_ratio)
    X_encoder_train, X_encoder_test, X_to_clas, y_to_clas = prepare_data_for_enc(X_train, y_train, autoenc_requared)
    X_train_classif, y_train_classif = prepare_data_for_classif(X_to_clas, y_to_clas, classif_requared)

    test_dl = make_dataloader(X_test, y_test)
    train_dl = make_dataloader(X_train_classif, y_train_classif)

    enc_train_dl = make_dataloader(X_encoder_train,encoder_data=True)
    enc_test_dl = make_dataloader(X_encoder_test,encoder_data=True)

    return train_dl, test_dl, enc_train_dl, enc_test_dl

In [9]:
# train_dl,\
# test_dl, enc_train_dl, enc_test_dl = prepare_data(df.drop(columns = ['Machine failure']),
#                                                   df['Machine failure'], 0.2, 6000, 300)

In [10]:
def train_cycle(df, list_amount = [50, 100, 200, 500, 1000, 1500, 1999]):
    for labels_amount in list_amount:
        ############################################################
        # PREPARE DATA
        ############################################################
        train_dl, test_dl, enc_train_dl, enc_test_dl = prepare_data(df.drop(columns = ['Machine failure']),
                                                                    df['Machine failure'], 0.2, 6000, labels_amount)

        ############################################################
        # TRAIN AUTOENCODER
        ############################################################
        # Set training settings
        enc_test_dl.dataset.data.shape[1]
        autoencoder = Autoencoder(enc_train_dl.dataset.data.shape[1], HIDDEN_PARAM, LATENT_REPR)
        loss = Loss_class_encoder(nn.MSELoss())
        model_factory = partial(Model_class)
        optimizer_factory = partial(torch.optim.AdamW)
        scheduler_factory = partial(lr.ExponentialLR)

        model_params = dict(model=autoencoder,
                            device=device)

        optimizer_params = dict(weight_decay=1e-3, lr=1e-2)
        scheduler_params = dict(gamma=0.95)

        learning_params = dict(batch_size=BATCH_SIZE, num_epoch=10)

        wandb_init_params = dict(
            name=f'Autoencoder_simple_HidParam-{HIDDEN_PARAM}_Latent-{LATENT_REPR}',
            project="Internship_project",
            dir = '../logs/'
        )
        # Start training
        trainer = Trainer(enc_train_dl,
                          enc_test_dl,
                          loss,
                          model_factory=model_factory,
                          optimizer_factory=optimizer_factory,
                          scheduler_factory=scheduler_factory,
                          model_params=model_params,
                          optimizer_params=optimizer_params,
                          scheduler_params=scheduler_params,
                          log=False,
                          wandb_init_params=wandb_init_params,
                          model_dir='../logs/nn_models/autoencoder/',
                          saving_model=False
                          )
        trainer.train_model(learning_params)
        wandb.finish()

        ############################################################
        # TRAIN CLASSIFIER
        ############################################################
        
        classifier = Simple_classifier(train_dl.dataset.data.shape[1], 10)
        jm = JoinedModel(trainer.model.model.encoder, classifier)

        loss = Loss_class(FocalLoss(gamma=2))
        model_factory = partial(Model_class)
        optimizer_factory = partial(torch.optim.AdamW)
        scheduler_factory = partial(lr.ExponentialLR)

        model_params = dict(model=jm,
                            device=device)

        optimizer_params = dict(weight_decay=1e-3, lr=1e-2)
        scheduler_params = dict(gamma=0.95)

        learning_params = dict(batch_size=BATCH_SIZE, num_epoch=40)

        wandb_init_params = dict(
            name=f'JM_NumLab-{labels_amount}_LatDim-{LATENT_REPR}',
            project="Internship_project",
            dir = '../logs/'
        )
        
        trainer = Trainer(train_dl,
                          test_dl,
                          loss,
                          model_factory=model_factory,
                          optimizer_factory=optimizer_factory,
                          scheduler_factory=scheduler_factory,
                          model_params=model_params,
                          optimizer_params=optimizer_params,
                          scheduler_params=scheduler_params,
                          log=True,
                          wandb_init_params=wandb_init_params,
                          model_dir='../logs/nn_models/joined_models/',
                          saving_model=False
                          )
        
        trainer.train_model(learning_params)
        wandb.finish()

In [11]:
train_cycle(df)

Let's see how good I am...: 100%|██████████| 1/1 [00:00<00:00, 105.92it/s]


Epoch: 1 of 10, 0.002 min


Let's see how good I am...: 100%|██████████| 1/1 [00:00<00:00, 104.81it/s]


Epoch: 2 of 10, 0.003 min


Let's see how good I am...: 100%|██████████| 1/1 [00:00<00:00, 103.70it/s]


...

Epoch: 38 of 40, 0.006 min


Let's see how good I am...: 100%|██████████| 2/2 [00:00<00:00, 29.38it/s]


Epoch: 39 of 40, 0.006 min


Let's see how good I am...: 100%|██████████| 2/2 [00:00<00:00, 34.63it/s]


Epoch: 40 of 40, 0.009 min
